In [18]:
# Always reload modules to have the current version
%reload_ext autoreload
%autoreload 2

In [19]:
from ranking.util import json_lines as jl
import pandas as pd

input_file = 'raw.docs.jsonl'
output_file  = 'all-unique-functions.jsonl'

In [20]:
def strip_extra_spaces(text):
    return " ".join(text.split())

def equalize_docItem(docItem):
    no_empty_ctx = "".join(docItem.split('() =>'))
    return strip_extra_spaces(no_empty_ctx)

sign = '($$!) :: () => (i -> r) -> Number r i -> r'
exptectedSign = '($$!) :: (i -> r) -> Number r i -> r'

assert(equalize_docItem(sign) == exptectedSign)
assert(exptectedSign == equalize_docItem(exptectedSign))

In [21]:
def get_unique_functions(finput, foutput):
    df = jl.read_jsonl(finput)
    df['equalizedDocItem'] = df.apply(
        lambda row: equalize_docItem(row['docItem']), axis=1)
    groups = df.groupby(['equalizedDocItem'])

    docIds = []
    docContents = []
    docItems = []
    docTypes = []

    for name, group in groups:
        if not name:
            # skip group that contains no functions e.g. packages, modules, ...
            continue
        rep = group.head(1).iloc[0]
        docIds.append(sorted(list(group['docId'])))
        docContents.append(rep['docContent'])
        docItems.append(rep['docItem'])
        docTypes.append(rep['docType'])

    res_df = pd.DataFrame({'docIds': docIds, 'docContent': docContents,
                           'docItem': docItems, 'docType': docTypes})
    jl.to_jsonl(res_df, foutput)


In [22]:
get_unique_functions(input_file, output_file)